In [1]:
import os, sys
import os, sys
project_dir = os.getcwd()
if project_dir not in sys.path:
    sys.path.append(project_dir)

import numpy as np
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt
from spectral.algorithms import spectral_angles
from dataset import Dataset

In [2]:
dataset_dir = '/home/abian/Data/Dataset/IUMA/DermaDatabase/'
dataset = Dataset(dataset_dir)
x, y = dataset.get()

In [3]:
classes = np.unique(y)
for i in range(len(classes)):
    idx = np.where(y==classes[i])
    n_samples = y[idx].size
    print('Class {}: {} samples'.format(classes[i], n_samples))

Class b'a': 13084 samples
Class b'b': 3051 samples
Class b'c': 1083 samples
Class b'd': 63 samples
Class b'e': 283 samples
Class b'f': 569 samples
Class b'g': 274 samples
Class b'h': 98 samples
Class b'i': 582 samples
Class b'j': 1378 samples
Class b'k': 90 samples
Class b'l': 3885 samples
Class b'm': 601 samples
Class b'n': 3775 samples
Class b'o': 73 samples
Class b'p': 132 samples
Class b'q': 24 samples
Class b'r': 244 samples


In [12]:
dataset.label_map

{'a': 100,
 'b': 200,
 'c': 201,
 'd': 202,
 'e': 203,
 'f': 204,
 'g': 207,
 'h': 209,
 'i': 210,
 'j': 300,
 'k': 301,
 'l': 400,
 'm': 401,
 'n': 402,
 'o': 403,
 'p': 404,
 'q': 406,
 'r': 407}

# Data Preprocessing

## Calibration 

$\begin{align}
    C_i = 100 * \frac{R_i - D_r}{W_r - D_r}
\end{align}$

where $C_i$ is the calibrated image, $R_i$ note raw image and the $W_r$ and $D_r$ represents the white and dark reference image, respectively.


In [4]:
# W_r y D_r es una imagen
def calibrate(img, w_r, d_r):
    if not(w_r.shape == d_r.shape == img.shape):
        assert('Dimensionality error')
    
    return 100  * (img - d_r) / (w_r - d_r)

# Normalize data
$\begin{align}
    P'_i = \frac{P_i - P_{min}}{P_{max} - P_{min}}
\end{align}$

where $P'_i$ is the normalized pixel value, $P_i$ the reflectance of the pixel, $P_{min}$ and $P_{max}$ is the minimum and maximum reflectance value, respectively.

In [5]:
def normalize(img):
    return (img - img.min()) / (img.max() - img.min())

# Sampling Interval Analysis

Selected band equidistant...

$\begin{align}
    Sampling Interval (nm) = \frac{\lambda_{max} - \lambda_{min}}{N_{\lambda}}
\end{align}$

where $\lambda_{max} - \lambda_{min}$ is the difference between the mamum and minimum wavelength and $N_{\lambda}$ is the number of band captured by the sensor.

In [6]:
def sampling_interval(lambda_min, lambda_max, n_spectral_bands):
    '''
        Param:
        -----
            lambda_min (int): minimum wavelenght
            lambda_max (int): maximum wavelenght
            n_spectral_bands (int): number of spectral bands captured by the sensor
    '''
    return (lambda_max - lambda_min) / n_spectral_bands

# Dataset reduction

**Spectral Angle Mapper**

$[\alpha = cos^{-1}\left ( \frac{\sum_{i = 1}^{nb} t_{i} r_{i}}{(\sum_{i = 1}^{nb} t_{i}^2)^{\frac{1}{2}} (\sum_{i = 1}^{nb} r_{i}^2)^{\frac{1}{2}}} \right )]$

where

* $\alpha$ = spectral angle between the standard and the spectral curve of the pixel
* $nb$ = number of spectral channels
* $t$ = vector of spectral response of the standard
* $r$ = the spectral response vector of the analyzed pixel

In [7]:
from spectral.algorithms import spectral_angles
from sklearn.cluster import KMeans

def spectral_angles_pixel(x, ref):
    '''
        For pixel input, the original function is prepare for image
    '''
    return spectral_angles(x[np.newaxis,:], ref)[0]

def get_most_relevant_samples(x, centroid, n_samples_per_centroid=10):
    '''
        Paper: Most Relevant Spectral Bands Identification for Brain
        Cancer Detection Using Hyperspectral Imaging    
    '''
    if len(x.shape) != 2:
        assert 'X shape error!'
    
    output = None
    result = spectral_angles_pixel(x, centroid)
    for i in range(len(centroid)):       
        ind = np.argpartition(result[i], -n_samples_per_centroid)[-n_samples_per_centroid:]
        if i == 0:
            output = x[ind]
        else:
            output = np.concatenate([output, x[ind]], axis=0)

    return output

def dataset_reduction(x, y, n_centroid_per_class=100, random_state=123):
    class_label = np.unique(y)
    final_x = None
    final_y = None
    for i in range(len(class_label)):
        idx = np.where(y==class_label[i])
        _x = x[idx]
        if len(idx[0]) > 1000:
            kmeans = KMeans(n_clusters=n_centroid_per_class, random_state=random_state).fit(_x)
            centroid = kmeans.cluster_centers_
            _x = get_most_relevant_samples(_x, centroid)

        if i == 0:
            final_x = _x
            final_y = np.full((_x.shape[0],), class_label[i])
        else:
            final_x = np.concatenate([final_x, _x], axis=0)
            final_y = np.concatenate([final_y, np.full((_x.shape[0],), class_label[i])], axis=0)
    
    return final_x, final_y

In [8]:
x_red, y_red = dataset_reduction(x, y)


In [9]:
x_red.shape

(9033, 116)

In [10]:
y_red.shape

(9033,)

In [ ]:
2**()

# Optimization

## Steps involved in HyperOptimization using Scikit-Optimizer

1. Define the space of hyperparameters to search
1. Define the function used to evaluate a given configuration
1. Minimize the loss using Space and Function defined in previous steps.

In [ ]:
import skopt

from feature_selection import FeatureSelection, FeatureEquidistantSelection
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Integer


# pipe = Pipeline([("transform", FeatureEquidistantSelection()), ('svc', SVC())])
pipe = Pipeline([("transform", FeatureSelection()), ('svc', SVC())])

params = dict()
n_features = x.shape[1]
# params['transform__n_features_to_select'] = (8, 34, 'uniform')
params['transform__selected_features'] = Integer(1, float(2**(116)-1), 'log-uniform')
params['svc__C'] = (1e-6, 100.0, 'log-uniform')
params['svc__gamma'] = (1e-6, 100.0, 'log-uniform')
params['svc__degree'] = (1,5)
params['svc__kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the search
search = BayesSearchCV(estimator=pipe, search_spaces=params, n_jobs=-1, cv=cv)
# perform the search
search.fit(x, y)
# report the best result
print(search.best_score_)
print(search.best_params_)

# Ant Colony Optimization (Testing)

**Buff, no furula. Buscar otra lib?**

probar **scikit-opt**: https://github.com/guofei9987/scikit-opt

https://www.youtube.com/watch?v=YFN_fJEu63w

In [ ]:
nodes = []
for _ in range(20):
  x = np.random.uniform(-10, 10)
  y = np.random.uniform(-10, 10)
  nodes.append((x, y))

nodes

In [ ]:
import pants
import math

def euclidean(a, b):
    return math.sqrt(pow(a[1] - b[1], 2) + pow(a[0] - b[0], 2))

In [ ]:
world = pants.World(nodes, euclidean)
solver = pants.Solver()
# solution = solver.solve(world)

solutions = solver.solutions(world)


In [ ]:
# print(solution.distance)
# print(solution.tour)    # Nodes visited in order
# print(len(solution.tour))
# print(solution.path)    # Edges taken in order
# print(len(solution.path))

best = float("inf")
for solution in solutions:
    assert solution.distance < best
    best = solution.distance
    print(best)
    print(len(solution.path))

print(best)

For each centroid, only the 10 most similar pixels are selected, having a total of 1000 pixels per class (100 centroids ×10 pixels). Thus, the reduced dataset is
intended to avoid the inclusion of redundant information in the training of the supervised classifier...

In [ ]:
solver.trace_elite[0]

In [ ]:
n_cluster = 2

X = np.array([[1, 2.5], [1, 4.1], [1, 0.1],
        [10, 2.1], [10, 4.9], [10, 0]])

kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(X)
centroid = kmeans.cluster_centers_
for x in X:
    plt.scatter(x=x[0], y=x[1], alpha=.2)

for x in centroid:
    plt.scatter(x=x[0], y=x[1], marker='*')

plt.show()